In [40]:
import accelerate
import datasets
from datasets import(
load_dataset, 
load_dataset_builder,
get_dataset_split_names,
get_dataset_config_names)
import peft
import transformers
from transformers import AutoTokenizer
# allows fast processing of datasets

In [33]:
ds_builder1=load_dataset_builder("LongSafari/open-genome", "stage1")
print(ds_builder1.info.features)
ds_builder2=load_dataset_builder("LongSafari/open-genome", "stage2")
print(ds_builder2.info.features)
ds_builder3=load_dataset_builder("LongSafari/open-genome", "sample")
print(ds_builder3.info.features)

{'record': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}


In [20]:
#this is how you load a dataset from the Hugging Face Hub
# the movie review data is for testing purposes
ds = load_dataset("cornell-movie-review-data/rotten_tomatoes", split="train")

In [37]:
ds_gs=load_dataset("LongSafari/open-genome", "sample")
get_dataset_split_names("LongSafari/open-genome", "sample")

['validation']

In [39]:
ds_gst1=load_dataset("LongSafari/open-genome", "stage1", split="train[:50]")
print(get_dataset_split_names("LongSafari/open-genome", "stage1"))
ds_gst2=load_dataset("LongSafari/open-genome", "stage2", split="train[:50]")
print(get_dataset_split_names("LongSafari/open-genome", "stage2"))

['train', 'validation', 'test']
['train', 'validation', 'test']


In [25]:
# this is used to obtain a dataset's split names
get_dataset_split_names("cornell-movie-review-data/rotten_tomatoes")

['train', 'validation', 'test']

In [29]:
configs=get_dataset_config_names("cornell-movie-review-data/rotten_tomatoes")
print(configs)
# returns all configs available to this dataset

['default']


In [42]:
# tokenizers process text and add padding/perform truncation for variables of inconsistent length
tokenizer_l = AutoTokenizer.from_pretrained("meta-llama/Llama-3.3-70B-Instruct")

In [45]:
tokenizer_m = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

In [54]:
def tokenize_l_function(examples):
    return tokenizer_l(examples["text"], padding="max_length", truncation=True)
# map applies preprocessing across a dataset
llama_tokenized_datasets=ds.map(tokenize_l_function, batched=True)

In [53]:
def tokenize_m_function(examples):
    return tokenizer_m(examples["text"], padding="max_length", truncation=True)

mistral_tokenized_datasets=ds.map(tokenize_m_function, batched=True)

Map: 100%|████████████| 8530/8530 [00:00<00:00, 22049.31 examples/s]


In [56]:
# perform preprocessing on the genomic data
def map_data(data, model):
    if model=="llama":
        tokenized_dataset=data.map(tokenize_l_function, batched=True)
    elif model=="mistral":
        tokenized_dataset=data.map(tokenize_m_function, batched=True)

In [58]:
l_tokenized_genome_samples=map_data(ds_gs, "llama")
m_tokenized_genome_samples=map_data(ds_gs, "mistral")

In [59]:
l_tokenized_genome_stage1=map_data(ds_gst1, "llama")
m_tokenized_genome_stage2=map_data(ds_gst2, "llama")

Map: 100%|███████████████████| 50/50 [00:03<00:00, 15.71 examples/s]


In [ ]:
# let's create smaller subsets of training datasets to fine-tune
